In [1]:
# importing required libraries:
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt
import pickle

c:\python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\python39\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\python39\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# setting the path to our eye dataset: 
Directory = r'C:\Users\KRISHNA\Drowsiness detection\dataset_new\train'
# specify two categories on which we want to train our data:
CATEGORIES = ['Closed' , 'Open']

In [3]:
#setting image size:
img_size = 24
data = []

#iterating over each image and get the image in array form,
for category in CATEGORIES:
    folder = os.path.join(Directory,category)
    label = CATEGORIES.index(category)
    for img in os.listdir(folder):
        img_path = os.path.join(folder, img)
        img_arr = cv2.imread(img_path)
        img_arr = cv2.cvtColor(img_arr, cv2.COLOR_BGR2GRAY)
        img_arr = cv2.resize(img_arr,(img_size, img_size),1)
        data.append([img_arr , label])

In [4]:
# see the length of data:
len(data)

1234

In [5]:
# we shuffle the data to get random images of open eyes and closed eyes:
random.shuffle(data)

In [6]:
# dividing features and label for training the model: 
X = []
Y = []

for features,label in data:
    X.append(features)
    Y.append(label)

In [7]:
#covert them into array:
X = np.array(X)
Y = np.array(Y)

In [8]:
# save the data into system:
pickle.dump(X , open('X.pkl' , 'wb'))
pickle.dump(Y , open('Y.pkl' , 'wb'))

In [9]:
# normalize the image array:
X = X/255

In [10]:
X

array([[[0.06666667, 0.0627451 , 0.10980392, ..., 0.69411765,
         0.72941176, 0.75294118],
        [0.05098039, 0.0745098 , 0.13333333, ..., 0.62352941,
         0.6627451 , 0.73333333],
        [0.05490196, 0.08235294, 0.12941176, ..., 0.49803922,
         0.6745098 , 0.6745098 ],
        ...,
        [0.31764706, 0.28235294, 0.29019608, ..., 0.29019608,
         0.30196078, 0.37647059],
        [0.31764706, 0.28627451, 0.30196078, ..., 0.30980392,
         0.39607843, 0.42352941],
        [0.33333333, 0.30196078, 0.30588235, ..., 0.36470588,
         0.44313725, 0.45882353]],

       [[0.56078431, 0.57254902, 0.58039216, ..., 0.40392157,
         0.44705882, 0.50588235],
        [0.54509804, 0.56078431, 0.55294118, ..., 0.4       ,
         0.41960784, 0.46666667],
        [0.54117647, 0.53333333, 0.54509804, ..., 0.39607843,
         0.38431373, 0.43137255],
        ...,
        [0.47058824, 0.50196078, 0.53333333, ..., 0.65098039,
         0.61960784, 0.70196078],
        [0.5

In [12]:
# reshape the X array to (24,24,1)
img_rows,img_cols = 24,24
X = X.reshape(X.shape[0],img_rows,img_cols,1)
X.shape

(1234, 24, 24, 1)

In [13]:
# we will be using keras to create the model:
from keras.models import Sequential
from keras.layers import Conv2D , MaxPooling2D , Flatten , Dense

In [14]:
# creating model:
model = Sequential()

model.add(Conv2D(64 , (3,3) , activation = 'relu' , input_shape= X.shape[1:]))
model.add(MaxPooling2D((1,1)))

model.add(Conv2D(64 , (3,3) , activation = 'relu'))
model.add(MaxPooling2D((1,1)))

model.add(Conv2D(64 , (3,3) , activation = 'relu'))
model.add(MaxPooling2D((1,1)))

model.add(Flatten())

model.add(Dense(128, activation = 'relu'))

model.add(Dense(2, activation = 'softmax'))

In [15]:
# compile model that we have created
model.compile(optimizer = 'adam' , loss = 'sparse_categorical_crossentropy' , metrics = ['accuracy'])

In [17]:
# fit X , Y to the model to see accuracy of model:
model.fit(X, Y, epochs = 5 , validation_split = 0.1 , batch_size = 32)

Epoch 1/5
35/35 [==============================] - 7s 201ms/step - loss: 0.0889 - accuracy: 0.9721 - val_loss: 0.2451 - val_accuracy: 0.9194
Epoch 2/5
35/35 [==============================] - 9s 245ms/step - loss: 0.0999 - accuracy: 0.9640 - val_loss: 0.1411 - val_accuracy: 0.9355
Epoch 3/5
35/35 [==============================] - 9s 257ms/step - loss: 0.0654 - accuracy: 0.9703 - val_loss: 0.1592 - val_accuracy: 0.9355
Epoch 4/5
35/35 [==============================] - 7s 212ms/step - loss: 0.0443 - accuracy: 0.9811 - val_loss: 0.1597 - val_accuracy: 0.9597
Epoch 5/5
35/35 [==============================] - 7s 203ms/step - loss: 0.0446 - accuracy: 0.9829 - val_loss: 0.2101 - val_accuracy: 0.9355


In [18]:
# save model and architecture to single file
model.save("custmodel.h5")